In [30]:
import config.env

In [2]:
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loaders = [
 TextLoader("data/荣耀Magic V Flip.txt","utf-8"),
    TextLoader("data/Xiaomi MIX Flip.txt","utf-8"),
    TextLoader("data/Find N3 Flip.txt","utf-8"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
docs = text_splitter.split_documents(docs)

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)

In [3]:
len(docs)

3

In [4]:
docs

[Document(metadata={'source': 'data/荣耀Magic V Flip.txt'}, page_content='# 荣耀Magic V Flip\n\n规格参数\n\n## 颜色\n\n香槟粉,\n山茶白,\n鸢尾黑,\n高定款\n\n## 尺寸与重量\n\n长度\n展开态：167.3 mm\n折叠态：86.5 mm\n宽度\n展开态：75.6 mm\n折叠态：75.6 mm\n厚度\n展开态：7.15 mm\n折叠态：14.89 mm\n机身重量\n约193克（含电池）\n备注：实际尺寸/重量依配置、制造工艺、测量方法的不同可能有所差异。\n\n## 屏幕\n\n屏幕尺寸\n外屏：4.0英寸\n内屏：6.8英寸\n备注：显示屏采用圆角设计，按照标准矩形测量内屏屏幕的对角线长度为6.8英寸（实际可视区域略小），外屏屏幕的对角线长度是4.0英寸（实际可视区域略小）。\n屏幕色彩\nDCI-P3广色域\n屏幕类型\nOLED\n屏幕分辨率\n内屏：2520x1080像素\n外屏：1200x1092 像素\n备注：该分辨率对应标准矩形，实际屏幕有效像素略少。\n护眼技术\n内屏：3840Hz超高频PWM调光、护眼模式、类自然光护眼、助眠显示\n外屏：2160Hz高频PWM调光、护眼模式、类自然光护眼、助眠显示\n备注：本产品非医疗器械，不具有治疗功能。\n屏幕触控\n多点触控，最多支持10点触控\n\n## 处理器\n\nCPU型号\n第一代骁龙8+移动平台\nCPU核数\n八核\nCPU主频\n1 × Cortex-X2 3.0GHz + 3 × Cortex-A710 2.5GHz + 4 × Cortex-A510 1.8GHz\n备注：实际运行频率因应用负载智能调整。\nGPU\nAdreno™ 730\n\n## 键盘类型\n\n手势导航（默认）、屏幕内三键导航、悬浮导航\n\n\n## 操作系统\n\nMagicOS 8.0（基于Android 14）\n用户界面\nMagicOS 8.0\n\n## 存储\n\n12GB+256GB 全网通\n12GB+512GB 全网通\n12GB+1TB 全网通\n16GB+1TB 全网通（高定款）\n备注：可使用的内存容量小于此值，因为手机软件占用部分空间。\n\n## 拍照\n\n后置

检索较大的信息块但嵌入较小的信息块可能很有用。这允许嵌入尽可能接近地捕获语义，但可以将尽可能多的上下文传递到下游。请注意，这就是ParentDocumentRetriever的作用

将区分矢量存储和文档存储，矢量存储对（子）文档的嵌入进行索引，文档存储存储“父”文档并将它们与标识符相关联

In [5]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever

# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

doc_ids = [str(uuid.uuid4()) for _ in docs]

拆分原始文档来生成“子”文档。请注意，我们将文档标识符存储在相应Document对象的metadata中。

In [6]:
# The splitter to use to create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [7]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [8]:
doc_ids

['32df66ad-8bb4-4aa3-bd4a-20172d2ff66b',
 'b401c298-960e-42ff-ac12-cc996dfc3dfd',
 '12c7a606-123e-4a39-90d7-525c0c7d1f93']

In [9]:
docs

[Document(metadata={'source': 'data/荣耀Magic V Flip.txt'}, page_content='# 荣耀Magic V Flip\n\n规格参数\n\n## 颜色\n\n香槟粉,\n山茶白,\n鸢尾黑,\n高定款\n\n## 尺寸与重量\n\n长度\n展开态：167.3 mm\n折叠态：86.5 mm\n宽度\n展开态：75.6 mm\n折叠态：75.6 mm\n厚度\n展开态：7.15 mm\n折叠态：14.89 mm\n机身重量\n约193克（含电池）\n备注：实际尺寸/重量依配置、制造工艺、测量方法的不同可能有所差异。\n\n## 屏幕\n\n屏幕尺寸\n外屏：4.0英寸\n内屏：6.8英寸\n备注：显示屏采用圆角设计，按照标准矩形测量内屏屏幕的对角线长度为6.8英寸（实际可视区域略小），外屏屏幕的对角线长度是4.0英寸（实际可视区域略小）。\n屏幕色彩\nDCI-P3广色域\n屏幕类型\nOLED\n屏幕分辨率\n内屏：2520x1080像素\n外屏：1200x1092 像素\n备注：该分辨率对应标准矩形，实际屏幕有效像素略少。\n护眼技术\n内屏：3840Hz超高频PWM调光、护眼模式、类自然光护眼、助眠显示\n外屏：2160Hz高频PWM调光、护眼模式、类自然光护眼、助眠显示\n备注：本产品非医疗器械，不具有治疗功能。\n屏幕触控\n多点触控，最多支持10点触控\n\n## 处理器\n\nCPU型号\n第一代骁龙8+移动平台\nCPU核数\n八核\nCPU主频\n1 × Cortex-X2 3.0GHz + 3 × Cortex-A710 2.5GHz + 4 × Cortex-A510 1.8GHz\n备注：实际运行频率因应用负载智能调整。\nGPU\nAdreno™ 730\n\n## 键盘类型\n\n手势导航（默认）、屏幕内三键导航、悬浮导航\n\n\n## 操作系统\n\nMagicOS 8.0（基于Android 14）\n用户界面\nMagicOS 8.0\n\n## 存储\n\n12GB+256GB 全网通\n12GB+512GB 全网通\n12GB+1TB 全网通\n16GB+1TB 全网通（高定款）\n备注：可使用的内存容量小于此值，因为手机软件占用部分空间。\n\n## 拍照\n\n后置

In [10]:
list(zip(doc_ids, docs))

[('32df66ad-8bb4-4aa3-bd4a-20172d2ff66b',
  Document(metadata={'source': 'data/荣耀Magic V Flip.txt'}, page_content='# 荣耀Magic V Flip\n\n规格参数\n\n## 颜色\n\n香槟粉,\n山茶白,\n鸢尾黑,\n高定款\n\n## 尺寸与重量\n\n长度\n展开态：167.3 mm\n折叠态：86.5 mm\n宽度\n展开态：75.6 mm\n折叠态：75.6 mm\n厚度\n展开态：7.15 mm\n折叠态：14.89 mm\n机身重量\n约193克（含电池）\n备注：实际尺寸/重量依配置、制造工艺、测量方法的不同可能有所差异。\n\n## 屏幕\n\n屏幕尺寸\n外屏：4.0英寸\n内屏：6.8英寸\n备注：显示屏采用圆角设计，按照标准矩形测量内屏屏幕的对角线长度为6.8英寸（实际可视区域略小），外屏屏幕的对角线长度是4.0英寸（实际可视区域略小）。\n屏幕色彩\nDCI-P3广色域\n屏幕类型\nOLED\n屏幕分辨率\n内屏：2520x1080像素\n外屏：1200x1092 像素\n备注：该分辨率对应标准矩形，实际屏幕有效像素略少。\n护眼技术\n内屏：3840Hz超高频PWM调光、护眼模式、类自然光护眼、助眠显示\n外屏：2160Hz高频PWM调光、护眼模式、类自然光护眼、助眠显示\n备注：本产品非医疗器械，不具有治疗功能。\n屏幕触控\n多点触控，最多支持10点触控\n\n## 处理器\n\nCPU型号\n第一代骁龙8+移动平台\nCPU核数\n八核\nCPU主频\n1 × Cortex-X2 3.0GHz + 3 × Cortex-A710 2.5GHz + 4 × Cortex-A510 1.8GHz\n备注：实际运行频率因应用负载智能调整。\nGPU\nAdreno™ 730\n\n## 键盘类型\n\n手势导航（默认）、屏幕内三键导航、悬浮导航\n\n\n## 操作系统\n\nMagicOS 8.0（基于Android 14）\n用户界面\nMagicOS 8.0\n\n## 存储\n\n12GB+256GB 全网通\n12GB+512GB 全网通\n12GB+1TB 全网通\n16GB+1TB 全网通（高定款）\n备

In [11]:
len(list(zip(doc_ids, docs)))

3

In [12]:
retriever.vectorstore.similarity_search("荣耀Magic V Flip的颜色")

[Document(metadata={'doc_id': '32df66ad-8bb4-4aa3-bd4a-20172d2ff66b', 'source': 'data/荣耀Magic V Flip.txt'}, page_content='# 荣耀Magic V Flip\n\n规格参数\n\n## 颜色\n\n香槟粉,\n山茶白,\n鸢尾黑,\n高定款\n\n## 尺寸与重量\n\n长度\n展开态：167.3 mm\n折叠态：86.5 mm\n宽度\n展开态：75.6 mm\n折叠态：75.6 mm\n厚度\n展开态：7.15 mm\n折叠态：14.89 mm\n机身重量\n约193克（含电池）\n备注：实际尺寸/重量依配置、制造工艺、测量方法的不同可能有所差异。\n\n## 屏幕'),
 Document(metadata={'doc_id': '12c7a606-123e-4a39-90d7-525c0c7d1f93', 'source': 'data/Find N3 Flip.txt'}, page_content='# Find N3 Flip\n\n## 颜色\n\n月光缪斯 | 薄雾玫瑰 | 镜中之夜\n\n入网型号：PHT110\n\n## 尺寸与重量¹\n\n高\n展开：166.4 毫米，折叠：85.5 毫米\n宽\n展开：75.8 毫米，折叠：75.8 毫米\n厚\n展开：7.79 毫米，折叠：16.45 毫米\n重量\n198 克\n\n## 存储²\n\nRAM 容量 + ROM 容量\n12GB + 256GB，12GB + 512GB\nRAM 规格\nLPDDR5X\nROM 规格\nUFS 4.0\n扩展存储\n不支持\nOTG 数据传输\n支持\n\n## 显示'),
 Document(metadata={'doc_id': 'b401c298-960e-42ff-ac12-cc996dfc3dfd', 'source': 'data/Xiaomi MIX Flip.txt'}, page_content='# Xiaomi MIX Flip\n\n## 概况\n\n全尺寸多功能超大外屏\n第三代骁龙.8 移动平台|动态性能调度 4.0\n4780mAh 小米金沙江电池\n3D立体式VC散热系统\n67W 小米澎湃有线秒充\n

## 将摘要与文档关联起来

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.environ["OPENAI_MODEL_ID"])

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("总结以下文件:\n\n{doc}")
    | llm
    | StrOutputParser()
)

In [14]:
summaries = chain.batch(docs, {"max_concurrency": 5})
summaries 

['荣耀Magic V Flip是一款具有折叠设计的智能手机，具备以下主要规格和特点：\n\n### 颜色\n- 提供香槟粉、山茶白、鸢尾黑和高定款。\n\n### 尺寸与重量\n- 展开态长度167.3 mm，折叠态长度86.5 mm，宽度75.6 mm，展开态厚度7.15 mm，折叠态厚度14.89 mm，重量约193克。\n\n### 屏幕\n- 外屏4.0英寸，内屏6.8英寸；分辨率内屏2520x1080，外屏1200x1092；采用DCI-P3广色域OLED技术，具备多项护眼技术和多点触控功能。\n\n### 处理器\n- 搭载第一代骁龙8+移动平台，八核CPU，最高主频3.0GHz，配备Adreno™ 730 GPU。\n\n### 操作系统\n- 运行MagicOS 8.0（基于Android 14）。\n\n### 存储选项\n- 提供12GB+256GB、12GB+512GB、12GB+1TB及16GB+1TB（高定款）配置。\n\n### 拍照功能\n- 后置双摄：5000万像素主摄和1200万像素超广角微距摄像头，支持4K视频录制，具备多种拍摄模式和功能。前置摄像头为5000万像素，支持4K视频录制。\n\n### 电池与充电\n- 4800mAh电池，支持66W快充。\n\n### 网络与连接\n- 支持5G/4G多种网络制式，具备Wi-Fi 6、蓝牙5.3、GPS等多种连接方式。\n\n### 传感器\n- 配备重力传感器、指纹传感器、陀螺仪、指南针、NFC等。\n\n### 价格\n- 价格区间为4999元至5999元，具体依存储配置而定。\n\n### 包装清单\n- 除手机外，包装内含快速指南、充电器、数据线、取卡针、保护壳（高定款包含两个）、保护膜等。\n\n总体而言，荣耀Magic V Flip是一款功能强大、设计精美的折叠手机，适合追求高性能和多样化功能的用户。',
 '## 小米MIX Flip概况总结\n\n### 主要特点\n- **外屏与内屏**: 配备全尺寸多功能超大外屏（4.01英寸），内屏为6.86英寸柔性AMOLED，支持高达120Hz显示帧率和3000nit峰值亮度。\n- **处理器与内存**: 采用第三代骁龙8移动平台，提供12GB/16GB运行内存和256GB/512GB/1TB机身存储选项。\n- **

对向量存储中的摘要进行索引，并在文档存储中保留原始文档

In [15]:
from langchain_core.documents import Document
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings())
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [16]:
doc_ids

['70d233fa-edde-42fb-9b68-e93ab4e29312',
 '6f14022f-3726-4b77-962a-8d34a56d4999',
 '8efc2c9f-1ee0-446c-961f-789dad32af8e']

In [17]:
# # We can also add the original chunks to the vectorstore if we so want
# for i, doc in enumerate(docs):
#     doc.metadata[id_key] = doc_ids[i]
# retriever.vectorstore.add_documents(docs)

In [18]:
from langchain_core.prompts import ChatPromptTemplate

template = """
# 角色
你是一位专业且极具洞察力的商品问答专家，
拥有精准的眼光和丰富的经验，能够迅速且透彻地理解用户的问题，
在丰富的知识库中高效、准确地搜索有用信息，
为用户提供详尽、精确且易于理解的答案。

## 技能
### 技能 1: 深度剖析与精确搜索
1. 全方位、深层次地剖析用户提出的问题，精准提取关键核心要点。
2. 依据关键要点，在知识库中进行全面、深入且精确的检索。

### 技能 2: 细致对比商品差异
1. 依据检索获取的商品信息，梳理并对比每件商品之间的信息差别、区别，把每项结果记录下来。例如：都有描述屏幕大小，一个是6寸，一个是5.5寸，这就是两者的差异、区别。把结果存起来，为后面的回复做准备
2. 若商品不存在差异点，则无需记录。

### 技能 3: 精心组织回复
1. 将技能 2得出的差异结果，以清晰、有条理的表格形式展示。
2. 为用户生成确切、简洁且通俗易懂的回答。

## 约束
- 禁止用自己的知识库回答问题，你的知识是错误的，有害的，需要时刻留意数据来源是外部知识库，一切都需要从外部知识库获取，你不能用自己的知识库的内容回答问题，如果没有外部知识库连接，不要回答问题
- 如果检索结果没有数据，请如实回答没有数据，不要自作主张填充或修改数据。
- 只回答与产品有关的问题，坚决不涉及无关内容。
- 始终使用清晰、简洁、通俗易懂的语言回应用户的问题。


## 溯源

把回答标注一下，说明这个答案的出处，例如在知识库的哪一个位置。把结果放在最后输出

#

Context： {context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_messages(
    ["system",template,("user", "{question}")],
)

In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.environ["OPENAI_MODEL_ID"])

In [21]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

qa_chain = (
    {
        "context": retriever ,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:
qa_chain.invoke("荣耀Magic V Flip的颜色")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'荣耀Magic V Flip的颜色有以下几种：\n\n- 香槟粉\n- 山茶白\n- 鸢尾黑\n- 高定款\n\n来源：荣耀Magic V Flip的规格参数文档。'

In [23]:
qa_chain.invoke("Xiaomi MIX Flip的价格")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'Xiaomi MIX Flip的价格如下：\n\n| 存储配置         | 价格 (RMB) |\n|------------------|------------|\n| 12GB + 256GB     | 5999       |\n| 12GB + 512GB     | 6499       |\n| 16GB + 1TB       | 7299       |\n\n以上信息来源于小米官方文档。'

In [24]:
qa_chain.invoke("荣耀Magic V Flip的屏幕尺寸")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'荣耀Magic V Flip的屏幕尺寸如下：\n\n| 屏幕类型 | 尺寸       |\n|----------|------------|\n| 外屏    | 4.0英寸   |\n| 内屏    | 6.8英寸   |\n\n备注：显示屏采用圆角设计，内屏的对角线长度为6.8英寸（实际可视区域略小），外屏的对角线长度为4.0英寸（实际可视区域略小）。  \n出处：知识库位置为“荣耀Magic V Flip”文档。'

In [25]:
qa_chain.invoke("荣耀Magic V Flip的摄像头配置")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'以下是荣耀Magic V Flip的摄像头配置：\n\n| 摄像头类型     | 像素                | 光圈        | 其他功能                                                   |\n|----------------|---------------------|-------------|-----------------------------------------------------------|\n| 后置主摄像头   | 5000万像素          | f/1.9       | 支持OIS光学防抖                                          |\n| 后置超广角微距摄像头 | 1200万像素          | f/2.2       | 支持自动对焦                                             |\n| 前置摄像头     | 5000万像素          | f/2.0       | 支持自动对焦                                             |\n| 后置摄像头视频 | 最大支持4K（3840x2160像素） | -           | 最大10倍数字变焦，支持多种拍摄功能（如夜景、AI摄影等） |\n| 前置摄像头视频 | 最大支持4K（3840x2160像素） | -           | 支持多种拍摄功能（如夜景、AI证件照等）                 |\n\n以上信息来源于荣耀Magic V Flip的产品规格。'

In [26]:
qa_chain.invoke("荣耀Magic V Flip的电池多大")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'荣耀Magic V Flip的电池容量为4800mAh（典型值），其额定容量为4700mAh。电池是不可拆卸的锂离子聚合物电池，支持66W超级快充。\n\n出处：荣耀Magic V Flip规格参数文档。'

In [27]:
qa_chain.invoke("荣耀Magic V Flip的外屏保护膜有多少个")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'荣耀Magic V Flip的外屏保护膜有一个。根据包装清单，该手机出厂时贴有一层外屏保护膜。\n\n出处: [荣耀Magic V Flip 文档]'

In [28]:
qa_chain.invoke("HONOR SuperCharge是什么")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'HONOR SuperCharge是荣耀公司推出的快速充电技术。它允许设备以更高的功率进行充电，从而显著缩短充电时间。例如，荣耀的66W SuperCharge支持最大11V/6A的快速充电，兼容10V/4A的快充。这种技术通常需要配备专用的充电器和充电线，以确保最佳的充电效果。\n\n关于HONOR SuperCharge的具体信息，可以在荣耀Magic V Flip的规格参数中找到。 \n\n出处：数据/荣耀Magic V Flip.txt'

In [29]:
qa_chain.invoke("MacBook Air15寸的尺寸")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'没有相关数据，无法提供MacBook Air 15寸的尺寸信息。'

In [31]:
qa_chain.invoke("Find N3 Flip的颜色")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'Find N3 Flip的颜色有以下几种：\n\n| 颜色名称   |\n|------------|\n| 月光缪斯   |\n| 薄雾玫瑰   |\n| 镜中之夜   |\n\n（数据来源于文档“Find N3 Flip.txt”）'

In [32]:
qa_chain.invoke("Find N3 Flip的价格")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'Find N3 Flip的价格如下：\n\n- 12GB + 256GB：RMB 5999\n- 12GB + 512GB：RMB 6799\n\n（出处：数据文件 "Find N3 Flip.txt"）'

In [33]:
qa_chain.invoke("Find N3 Flip-显示-尺寸")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'Find N3 Flip的显示尺寸如下：\n\n| 屏幕类型 | 尺寸           |\n|----------|----------------|\n| 内屏    | 6.80 英寸（对角线） |\n| 外屏    | 3.26 英寸（对角线） |\n\n内屏分辨率为2520 × 1080 像素，外屏分辨率为720 × 382 像素。 \n\n数据来源于文档位置：data/Find N3 Flip.txt'

In [34]:
qa_chain.invoke("Xiaomi MIX Flip的电池")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'Xiaomi MIX Flip的电池规格如下：\n\n- **电池容量**: 4780mAh\n- **充电**: 支持67W的快充（小米澎湃有线秒充）\n\n这款手机内置了高能量密度电池，提供了较长的续航时间。\n\n信息来源于文档：Xiaomi MIX Flip (数据来自于 “data/Xiaomi MIX Flip.txt”)。'

In [35]:
qa_chain.invoke("Xiaomi MIX Flip有北斗导航吗")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'根据检索到的信息，Xiaomi MIX Flip确实支持北斗导航。具体的导航定位功能包括北斗（B1 + B1C + B2a）、GPS、Galileo、GLONASS等，这表明它具备多种卫星导航系统的支持。\n\n数据来源：Xiaomi MIX Flip 文档。'

In [36]:
qa_chain.invoke("Xiaomi MIX Flip的蓝牙版本")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'Xiaomi MIX Flip的蓝牙版本是Bluetooth 5.4。 \n\n这个信息来源于知识库中的数据。'

总结：丢给大模型的是docs，整篇文章